In [ ]:
from google.colab import drive
drive.flush_and_unmount()
drive.mount('/content/drive', force_remount=True)
import sys
!ls /content/drive/MyDrive/'Colab Notebooks'/conformal-prediction-introduction
sys.path.append('/content/drive/MyDrive/Colab Notebooks/conformal-prediction-introduction')


In [ ]:
import torch
from torchvision import datasets, transforms, models
from src.data import IndexedDataset
from src.train import train_model, evaluate_and_save
 
import torch.nn as nn
from functools import partial


print("Running with",  "cuda" if torch.cuda.is_available() else "cpu")

# Hyperparameters
epochs = 20
lr = 1e-3
num_workers = 4
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results_folder = '/content/drive/MyDrive/Colab Notebooks/conformal-prediction-introduction/cifar100_results'

!mkdir -p {results_folder}


In [ ]:
# Data transformation and augmentation
transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.4914, 0.4822, 0.4465), (0.2023, 0.1994, 0.2010)),
])

# --- 1. DOWNLOAD CIFAR-100 --- 
cifar100_train = datasets.CIFAR100(root='./data', train=True, download=True, transform=transform_train)
cifar100_test = datasets.CIFAR100(root='./data', train=False, download=True, transform=transform_test)

# Split the training set into training, validation, and holdout/calibration sets
train_size = 40000 # 80% of 50000
val_size = 5000   # 10% of 50000
holdout_size = 5000 # 10% of 50000

trainset, valset, holdoutset, _ = torch.utils.data.random_split(cifar100_train, 
    [train_size, val_size, holdout_size, 0], 
    generator=torch.Generator().manual_seed(42)
)

testset = cifar100_test

# Wrap datasets to include index
trainset = IndexedDataset(trainset)
valset = IndexedDataset(valset)
testset = IndexedDataset(testset)
holdoutset = IndexedDataset(holdoutset)


dataloader_settings = partial(
    torch.utils.data.DataLoader,
    batch_size=64,
    num_workers=num_workers,
    pin_memory=True if device.type == "cuda" else False,
    persistent_workers=True,
)

trainloader = dataloader_settings(trainset, shuffle=True, drop_last=True)
valloader = dataloader_settings(valset, shuffle=False)
testloader = dataloader_settings(testset, shuffle=False)
holdoutloader = dataloader_settings(holdoutset, shuffle=False)

# Model setup
model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
# --- 2. UPDATE OUTPUT CLASSES (100) ---
model.fc = nn.Linear(model.fc.in_features, 100)
model = model.to(device)



# Train the model
model = train_model(model, trainloader, valloader, device, epochs=epochs, lr=lr)

# Evaluation on validation set
evaluate_and_save(model, valloader, device, results_folder, "val_predictions.pth")

# Evaluation on test set
evaluate_and_save(model, testloader, device, results_folder, "test_predictions.pth")

# Evaluation on holdout set (for calibration)
evaluate_and_save(model, holdoutloader, device, results_folder, "cal_predictions.pth")
